In [1]:
import os

# Path to your dataset (shown in your Kaggle screenshot)
root_dir = "/kaggle/input/pcb-defects/"

# Path to save trained models or logs
output_dir = "/kaggle/working/"

print("Dataset path:", root_dir)
print("Output path:", output_dir)


In [2]:
!pip install -U ultralytics

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

import os
import shutil
import random
import xml.etree.ElementTree as ET
import yaml
from pathlib import Path
from collections import Counter

from ultralytics import YOLO
from sklearn.model_selection import KFold

# Explore Dataset

In [4]:
dataset_dir = os.path.join(root_dir, 'PCB_DATASET')

for root, dirs, files in os.walk(dataset_dir):
   for name in dirs:
      print(os.path.join(root, name))

In [5]:
def count_files_in_folder(folder_path):
    # Get list of all files in the folder
    files = os.listdir(folder_path)

    # Count the number of files
    num_files = len(files)

    return num_files

In [6]:
subfolders = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']

images_dir = os.path.join(dataset_dir, 'images')
annot_dir = os.path.join(dataset_dir, 'Annotations')

for subfolder in subfolders:
    images_path = os.path.join(images_dir, subfolder)
    annot_path = os.path.join(annot_dir, subfolder)

    print(f'{subfolder:<15} \t\
            {count_files_in_folder(images_path)} images \t\
            {count_files_in_folder(annot_path)} annotations')

# Create annotaton dataset

In [7]:
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    data = []

    filename = root.find('filename').text
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    for obj in root.findall('object'):
        name = obj.find('name').text
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)

        data.append({
            'filename': filename,
            'width': width,
            'height': height,
            'class': name,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax
        })

    return data

In [8]:
# List to store parsed data from all XML files
all_data = []

# Recursively traverse subdirectories
for root, dirs, files in os.walk(annot_dir):
    for name in files:
        if name.endswith('.xml'):
            xml_path = os.path.join(root, name)
            all_data.extend(parse_xml(xml_path))

In [9]:
# Create DataFrame from the parsed data
annot_df = pd.DataFrame(all_data)
annot_df.head()

# Visualization

In [10]:
def get_subfolder(image_name):
    if 'missing' in image_name.split('_'):
        return 'Missing_hole'
    if 'mouse' in image_name.split('_'):
        return'Mouse_bite'
    if 'open' in image_name.split('_'):
        return 'Open_circuit'
    if 'short' in image_name.split('_'):
        return 'Short'
    if 'spur' in image_name.split('_'):
        return 'Spur'
    if 'spurious' in image_name.split('_'):
        return 'Spurious_copper'

In [11]:
def visualize_annotations(image_name, images_dir, annot_df, is_subfolder=False):
    # Construct path for image
    if is_subfolder:
        image_path = os.path.join(images_dir, get_subfolder(image_name), image_name)
    else:
        image_path = os.path.join(images_dir, image_name)


    # Read image
    image = cv2.imread(image_path)

    # Filter annotations for the current image
    annotations = annot_df[annot_df['filename'] == image_name]

    # Draw bounding boxes on the image
    for _, annot in annotations.iterrows():
        xmin, ymin, xmax, ymax = annot['xmin'], annot['ymin'], annot['xmax'], annot['ymax']
        class_label = annot['class']

        # Check if confidence column exists
        confidence = annot.get('confidence')
        if confidence is not None:
            class_label += f" ({confidence:.2f})"

        color = (255, 255, 255)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 3)

        # Add background to the text
        text_size = cv2.getTextSize(class_label, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 2)[0]
        cv2.rectangle(image, (xmin, ymin - text_size[1] - 5),
                             (xmin + text_size[0], ymin - 1), color, -1)

        # Add text
        cv2.putText(image, class_label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 2)

    # Convert BGR image to RGB (Matplotlib expects RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Plot the image with annotations
    plt.figure(figsize=(18, 10))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title('Annotations')
    plt.text(10, image_rgb.shape[0] + 100, f'Image: {image_name}',
             color='black', fontsize=11, ha='left')
    plt.show()

    return image


In [12]:
image_name = '04_short_03.jpg'
visualize_annotations(image_name, images_dir, annot_df, is_subfolder=True);

# Dataset preprocessing

In [13]:
def resize_images(input_dir, output_dir, target_size=(768, 768)):
    os.makedirs(output_dir, exist_ok=True)

    target_w, target_h = target_size

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith('.jpg'):
                image_path = os.path.join(root, file)
                img = cv2.imread(image_path)

                h, w = img.shape[:2]
                scale = min(target_w / w, target_h / h)

                new_w = int(w * scale)
                new_h = int(h * scale)

                resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

                canvas = np.full((target_h, target_w, 3), 114, dtype=np.uint8)
                canvas[:new_h, :new_w] = resized

                output_path = os.path.join(output_dir, file)
                cv2.imwrite(output_path, canvas)


# YOU MUST SAVE IN A WRITABLE DIRECTORY
resized_img_dir = "/kaggle/working/images_resized"

resize_images(images_dir, resized_img_dir)


In [14]:
def resize_annotations(annot_df, target_size=(768, 768)):
    target_w, target_h = target_size
    all_data = []

    for _, row in annot_df.iterrows():
        orig_w = row['width']
        orig_h = row['height']

        # same scale factor used in resize_images()
        scale = min(target_w / orig_w, target_h / orig_h)

        # no padding offset because we place image at top-left (0,0)
        pad_x = 0
        pad_y = 0

        # apply scale
        xmin = int(row['xmin'] * scale + pad_x)
        ymin = int(row['ymin'] * scale + pad_y)
        xmax = int(row['xmax'] * scale + pad_x)
        ymax = int(row['ymax'] * scale + pad_y)

        all_data.append({
            'filename': row['filename'],
            'width': target_w,
            'height': target_h,
            'class': row['class'],
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax
        })

    return pd.DataFrame(all_data)


annot_df_resized = resize_annotations(annot_df)
annot_df_resized.head()

# Split dataset

In [15]:
# Create the output directory (MUST BE IN kaggle/working/)
output_dir = "/kaggle/working/output"
os.makedirs(output_dir, exist_ok=True)

print("Output directory:", output_dir)


In [16]:
# Convert annotation DataFrame into YOLO labels
# YOLO format: <class_index> <x_center> <y_center> <width> <height>
def convert_to_yolo_labels(annotation_df, classes, target_size=(768, 768)):
    yolo_labels = []

    for _, annot in annotation_df.iterrows():
        filename = annot['filename']
        width, height = annot['width'], annot['height']
        class_name = annot['class']
        xmin, ymin, xmax, ymax = annot['xmin'], annot['ymin'], annot['xmax'], annot['ymax']

        # Convert bounding box coordinates to YOLO format
        x_center = (xmin + xmax) / (2 * width)
        y_center = (ymin + ymax) / (2 * height)
        bbox_width = (xmax - xmin) / width
        bbox_height = (ymax - ymin) / height

        class_index = classes.index(class_name)

        # Append to YOLO labels list
        yolo_labels.append((filename, class_index, x_center, y_center, bbox_width, bbox_height))

    return yolo_labels


classes = ['missing_hole', 'mouse_bite', 'open_circuit',
           'short', 'spur', 'spurious_copper']
yolo_labels = convert_to_yolo_labels(annot_df_resized, classes)


In [17]:
def split_images_and_labels(images_dir, labels, output_dir, train_split=0.95, val_split=0.05):
    # os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/test'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/test'), exist_ok=True)

    # Group labels by image filename
    image_labels = {}
    for label in labels:
        filename, class_index, x_center, y_center, bbox_width, bbox_height = label
        if filename not in image_labels:
            image_labels[filename] = []
        image_labels[filename].append(label)

    # Shuffle the image filenames
    image_filenames = list(image_labels.keys())
    random.shuffle(image_filenames)

    # Split the dataset
    num_images = len(image_filenames)
    num_train = int(num_images * train_split)
    num_val = int(num_images * val_split)

    train_filenames = image_filenames[:num_train]
    val_filenames = image_filenames[num_train:num_train + num_val]
    test_filenames = image_filenames[num_train + num_val:]

    # Write train, val, test images and labels
    for dataset, filenames in [('train', train_filenames), ('val', val_filenames), ('test', test_filenames)]:
        for filename in filenames:
            labels = image_labels[filename]
            with open(os.path.join(output_dir, f'labels/{dataset}/{os.path.splitext(filename)[0]}.txt'), 'w') as label_file:
                for label in labels:
                    _, class_index, x_center, y_center, bbox_width, bbox_height = label
                    label_file.write(f"{class_index} {x_center} {y_center} {bbox_width} {bbox_height}\n")
            # Copy images to corresponding folders
            shutil.copy(os.path.join(images_dir, filename), os.path.join(output_dir, f'images/{dataset}/{filename}'))

split_images_and_labels(resized_img_dir, yolo_labels, output_dir)

# K-Fold Cross Validation

In [18]:
dataset_path = Path(output_dir)
labels = sorted(dataset_path.rglob("*labels/train/*.txt")) # all data in 'labels'

In [19]:
cls_idx = list(range(len(classes)))
print(list(zip(classes, cls_idx)))

In [20]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [21]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`
labels_df.head()

In [22]:
ksplit = 3
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [23]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

In [24]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

fold_lbl_distrb

In [25]:
# Initialize a list to store image file paths
images = sorted(dataset_path.rglob("*images/train/*.jpg"))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f'{ksplit}fold_crossval')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

In [26]:
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [27]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

# Model Training

In [28]:
from ultralytics import YOLO
import torch, gc

results = {}

model_name  = "yolov8m.pt"
batch       = 8
epochs      = 180
imgsz       = 768
save_period = 5
project     = "pcb_m_sota_v8_clean_fixed"

for k in range(ksplit):
    print(f"\n🔵 TRAINING FOLD {k+1}/{ksplit}\n")

    model = YOLO(model_name)
    dataset_yaml = ds_yamls[k]

    model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch,
        imgsz=imgsz,

        # ---- LEARNING ----
        lr0=0.002,
        lrf=0.0001,
        momentum=0.95,
        weight_decay=0.0005,
        warmup_epochs=4,
        warmup_momentum=0.9,

        # ---- LOSS ----
        box=7.0,
        cls=1.0,

        # ---- AUGMENTATION ----
        mosaic=0.8,
        mixup=0.0,
        erasing=0.35,
        copy_paste=0.15,
        auto_augment="randaugment",
        hsv_h=0.02,
        hsv_s=0.8,
        hsv_v=0.5,
        translate=0.12,
        scale=0.55,
        fliplr=0.5,
        perspective=0.0005,

        # ---- DETECTION ----
        multi_scale=False,   # 🔥 IMPORTANT FIX
        iou=0.2,
        conf=0.001,

        # ---- SYSTEM ----
        amp=True,
        workers=0,           # 🔥 IMPORTANT FIX
        cache=False,

        # ---- SAVE / LOG ----
        project=project,
        name=f"fold_{k}",
        save_period=save_period,
        verbose=True,
    )

    results[k] = model.metrics

    del model
    gc.collect()
    torch.cuda.empty_cache()


In [38]:
from ultralytics import YOLO

# Add '_fixed' to the path
best_model = YOLO("/kaggle/working/pcb_m_sota_v8_clean_fixed/fold_2/weights/best.pt")
best_model.val(imgsz=768, augment=True)

# Model

In [41]:
pip install -U ipywidgets

In [44]:
result = model.train(
    data=data_path,
    epochs=epochs,
    batch=batch,
    imgsz=imgsz,

    # ---------- Optimized learning ----------
    lr0=0.002,
    lrf=0.0001,
    momentum=0.95,
    weight_decay=0.0005,
    warmup_epochs=4,
    warmup_momentum=0.9,

    # ---------- Loss ----------
    box=7.0,
    cls=1.0,

    # ---------- Augmentation ----------
    mosaic=0.9,
    mixup=0.0,
    erasing=0.35,
    copy_paste=0.15,
    auto_augment="randaugment",
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.5,
    translate=0.12,
    scale=0.55,
    fliplr=0.5,
    perspective=0.0005,

    # ---------- Detection ----------
    iou=0.2,
    conf=0.001,

    # ---------- Kaggle-safe ----------
    amp=True,
    device=0,
    cache=False,

    # ---------- Logging ----------
    project=project,
    name="run_m_sota",
    save_period=save_period,
    verbose=verbose,
)


In [46]:
results_dir = "/kaggle/working/pcb_m_single_sota/run_m_sota"

# SAVE RESULTS HERE (WRITEABLE)
dest_results_dir = "/kaggle/working/results"

os.makedirs(dest_results_dir, exist_ok=True)

shutil.copytree(results_dir, dest_results_dir, dirs_exist_ok=True)


In [47]:
results_df = pd.read_csv(os.path.join(dest_results_dir, 'results.csv'))
results_df.columns = results_df.columns.str.strip()
results_df = results_df.apply(pd.to_numeric, errors='coerce').dropna()
results_df.head()

In [48]:
epochs = results_df['epoch']
train_box_loss = results_df['train/box_loss']
val_box_loss = results_df['val/box_loss']
train_cls_loss = results_df['train/cls_loss']
val_cls_loss = results_df['val/cls_loss']
train_dfl_loss = results_df['train/dfl_loss']
val_dfl_loss = results_df['val/dfl_loss']

# Create a figure with three subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot box loss
axs[0].plot(epochs, train_box_loss, label='Train Box Loss', color='blue')
axs[0].plot(epochs, val_box_loss, label='Validation Box Loss', color='orange')
axs[0].set_title('Box Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)

# Plot cls loss
axs[1].plot(epochs, train_cls_loss, label='Train Cls Loss', color='blue')
axs[1].plot(epochs, val_cls_loss, label='Validation Cls Loss', color='orange')
axs[1].set_title('Class Loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].legend()
axs[1].grid(True)

# Plot dfl loss
axs[2].plot(epochs, train_dfl_loss, label='Train Dfl Loss', color='blue')
axs[2].plot(epochs, val_dfl_loss, label='Validation Dfl Loss', color='orange')
axs[2].set_title('Distribution Focal Loss')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('Loss')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()

# Predict on test data

In [73]:
# ------------------------------
# Load the trained model
# ------------------------------
train_dir = "/kaggle/working/pcb_m_single_sota/run_m_sota"
best_model_path = os.path.join(train_dir, "weights/best.pt")
model = YOLO(best_model_path)

print("Using model:", best_model_path)

# ------------------------------
# Run inference on val images
# ------------------------------
test_data_dir = os.path.join(output_dir, "images/val")

metrics = model(source=test_data_dir, imgsz=640, conf=0.25, save=True, save_txt=True, save_conf=True)

In [77]:
predict_dir = "/kaggle/working/runs/detect/predict"

# -----------------------------------
# Save predictions safely in working/
# -----------------------------------
dest_predict_dir = "/kaggle/working/results/predict2"

os.makedirs(dest_predict_dir, exist_ok=True)

shutil.copytree(predict_dir, dest_predict_dir, dirs_exist_ok=True)

print("✔ Predictions saved to:", dest_predict_dir)

best_model_path = os.path.join(dest_results_dir, 'weights/best.pt')
model = YOLO(best_model_path)

test_data_dir = os.path.join(output_dir, 'images/val')
metrics = model(source=test_data_dir, imgsz=640, conf=0.25, save=True, save_txt=True, save_conf=True)

In [78]:
def yolo_to_original_annot(image_name, yolo_labels, annot_df, classes):
    original_annot = []

    for yolo_label in yolo_labels:
        # Extract original width and height from annotation DataFrame
        original_size = annot_df.loc[annot_df['filename'] == image_name, ['width', 'height']].iloc[0]
        original_width, original_height = original_size['width'], original_size['height']

        # Extract YOLO label components
        class_index, x_center, y_center, bbox_width, bbox_height, confidence = yolo_label

        # Scale bounding box coordinates and dimensions to original size
        original_x_center = x_center * original_width
        original_y_center = y_center * original_height
        original_bbox_width = bbox_width * original_width
        original_bbox_height = bbox_height * original_height

        # Calculate original bounding box coordinates
        original_x_min = original_x_center - original_bbox_width / 2
        original_y_min = original_y_center - original_bbox_height / 2
        original_x_max = original_x_center + original_bbox_width / 2
        original_y_max = original_y_center + original_bbox_height / 2

        # Append original annotation to list
        original_annot.append({
            'filename': image_name,
            'width': int(original_width),
            'height': int(original_height),
            'class': classes[int(class_index)],
            'xmin': int(original_x_min),
            'ymin': int(original_y_min),
            'xmax': int(original_x_max),
            'ymax': int(original_y_max),
            'confidence': confidence
        })

    return pd.DataFrame(original_annot)

In [93]:
def read_yolo_labels_from_file(file_path):
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            values = line.strip().split()
            values = [float(value) for value in values]
            labels.append(values)
    return labels

file_path = os.path.join('/kaggle/working/runs/detect/predict4', 'labels/11_missing_hole_02.txt')
yolo_labels = read_yolo_labels_from_file(file_path)

yolo_labels

In [94]:
pred_annot_df = yolo_to_original_annot('11_missing_hole_02.jpg', yolo_labels, annot_df, classes)
pred_annot_df.head()

In [95]:
visualize_annotations('11_missing_hole_02.jpg', images_dir, pred_annot_df, is_subfolder=True);

In [97]:
model.export()

In [100]:
import os
from IPython.display import FileLink

# 1. CLEANUP: Delete ALL old zip files to free up massive space
print("🧹 Cleaning up old zip files...")
!rm -f *.zip
!rm -f *.7z
print("✅ Cleanup done.")

# 2. ZIP EVERYTHING: 'output', 'runs', and 'results' together
# We use -q (quiet) to prevent freezing
# We exclude 'predict' images from runs if they are huge to save space (Optional)
print("📦 Zipping 'output', 'runs', and 'results'... this may take 2-3 minutes.")

# This command zips all 3 folders into 'final_data.zip'
!zip -r -q final_data.zip output runs results

# 3. VERIFY & DOWNLOAD
if os.path.exists("final_data.zip"):
    size_mb = os.path.getsize("final_data.zip") / (1024 * 1024)
    print(f"\n✅ SUCCESS! Zip created ({size_mb:.2f} MB).")
    print("Click the link below to download:")
    display(FileLink('final_data.zip'))
else:
    print("\n❌ Error: Zip creation failed. You might be completely out of space.")
    print("Try deleting the 'runs' folder if you don't need the prediction images.")

# Code below to test custom images

In [68]:
# Input images (still located inside /kaggle/input/)
custom_img_dir = os.path.join(root_dir, 'PCB_DATASET/custom_images')

# Output must be inside /kaggle/working/
resized_custom_img_dir = '/kaggle/working/custom_images_resize'

resize_images(custom_img_dir, resized_custom_img_dir)


In [70]:
model = YOLO('/kaggle/working/pcb_m_single_sota/run_m_sota/weights/best.torchscript')
01.jpg')

image_path = os.path.join(resized_custom_img_dir, 'result_custom = model(image_path, imgsz=768, conf=0.25, save=True, save_txt=True, save_conf=True)

In [ ]:
predict_dir = '/content/runs/detect/predict2'
dest_custom_predict_dir = os.path.join(custom_img_dir, 'results/predict')

shutil.copytree(predict_dir, dest_custom_predict_dir)

# Gradio Gui

In [98]:
import gradio as gr
from ultralytics import YOLO
import cv2
import os

# === Load trained YOLOv8 model ===
model_path = '/kaggle/working/results/weights/best.torchscript'  # update if needed
assert os.path.exists(model_path), f"❌ Model not found at {model_path}"
model = YOLO(model_path)

# === Detection function ===
def detect_pcb_defects(image):
    # Run YOLOv8 inference
    results = model.predict(source=image, save=False, conf=0.15, imgsz=768)
    # Extract annotated image
    annotated_img = results[0].plot()  # returns numpy array (BGR)
    
    # Convert BGR → RGB for display
    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    return annotated_img

# === Gradio UI ===
title = "🔍 PCB Defect Detection using YOLOv8"
description = (
    "Upload a PCB image to detect defects such as missing holes, open circuits, "
    "mouse bites, shorts, and spurious copper using a trained YOLOv8 model."
)

demo = gr.Interface(
    fn=detect_pcb_defects,
    inputs=gr.Image(type="filepath", label="Upload PCB Image"),
    outputs=gr.Image(label="Detection Result"),
    title=title,
    description=description,
    examples=[
        ["/home/arm/Downloads/PCB/example_images/pcb1.png"],
        ["/home/arm/Downloads/PCB/example_images/pcb2.png"]
    ],
)

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=None)